- 2nd column, matches the faculty naming pattern. That's where the index starts. Automate it this way. 
- Remove the first 0 in the values of section rows if there are any.

In [ ]:
from scripts.script import faculty_df, course_catalog, faculty_finder

In [37]:
import pandas as pd

In [67]:
df = pd.read_excel("../dataset/spring_data.xlsx")

In [ ]:
df

In [68]:
def verify_faculty(s):
    words = s.split()
    return len(words) == 1 and words[0].isalpha()

def faculty_df(df): 
    df.drop(df.columns[0], axis=1, inplace=True)
    df = pd.concat([df[column] for column in df.columns], ignore_index=True)
    df.dropna(inplace=True)
    df = df.str.split('-', n=1, expand=True)
    df.iloc[:, 0] = df.iloc[:, 0].apply(lambda s: s.strip())
    df = df[df.iloc[:, 0].apply(lambda s: verify_faculty(s))]
    df.rename(columns={0: 'code', 1: 'name'}, inplace = True)
    df.loc[:, 'code'] = df.loc[:, 'code'].apply(lambda s: s.strip())
    df.loc[:, 'name'] = df.loc[:, 'name'].apply(lambda s: s.strip())

    new_row = pd.DataFrame([{'code': 'TBA', 'name': 'To be announced'}])
    df = pd.concat([df, new_row], ignore_index=True)
    
    return df 
    

In [69]:
def course_catalog (df):
    df = df.drop(df.columns[0], axis = 1)
    df = pd.concat([df[column] for column in df.columns], ignore_index=True)
    df = df.dropna()
    pattern = r'([A-Z]{3} \d{3}.*)'
    df = df.str.extract(pattern).dropna()
    df = df.iloc[:, 0].str.split('-', expand = True)
    df.iloc[:, 3] = df.iloc[:, 3].apply(lambda s: s.split()[0])
    df = df.drop(df.columns[[2,4]], axis = 1)
    df.rename(columns={0: 'cc',
                       1: 'section',
                       3: 'faculty'},
              inplace = True)
    df = df.map(lambda s: s.strip())
    df.loc[:, 'section'] = df.loc[:, 'section'].apply(lambda s: s.lstrip('0'))
    
    return df
    
    

In [70]:
def faculty_finder(courses: pd.DataFrame,
                   faculty: pd.DataFrame,
                   cc: str,
                   sec: str):
    row = courses[(courses['cc'] == cc) & (courses['section'] == sec)].loc[:, 'faculty']
    row = row.iloc[0]
    fac = faculty.loc[faculty.loc[:, 'code'] == row].loc[:, 'name'].item()
    
    return fac

In [71]:
faculty = faculty_df(df)
courses = course_catalog(df)
cc = input('Enter course code: ')
sec = input('Enter section: ')

In [72]:
faculty_finder(courses, faculty, cc, sec)

'Nafees Monsoor'

In [52]:
courses.head()

,cc,section,faculty
3,BUS 1201,7,NMK
4,BUS 1201,9,EAZ
5,BUS 2101,2,GAU
6,BUS 2201,1,FMS
7,BUS 2301,5,GMSU


In [59]:
courses[courses['section'] == '2']

,cc,section,faculty
5,BUS 2101,2,GAU
8,BUS 3102,2,ShS
13,CSE 1203,2,TBA
18,CSE 2301,2,JFR
22,CSE 3201,2,TBA
...,...,...,...
897,PHY 1101,2,TBA
916,BUS 3201,2,MinUA
917,BUS 3202,2,ZAh
937,ENG 2203,2,JNT


In [57]:
courses.loc[:, 'section'] = courses.loc[:, 'section'].apply(lambda s: s.lstrip('0'))

In [ ]:
courses